In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
# from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
from torchvision import datasets, transforms
from torchviz import make_dot

import os
import time
import random
import networkx as nx
import yaml
import matplotlib.pyplot as plt

In [2]:
import os
import time
import random
import networkx as nx
import yaml
import pandas as pd
import shutil
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Resize, ToTensor, Normalize, transforms
from torchvision.datasets import ImageFolder
from zlib import crc32


use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [3]:
original_dataset_dir = './input/data/train/images'
base_dir = './splitted'
classes_list = ["{0:02d}".format(a) for a in range(18)]

if not os.path.isdir(base_dir): 
    os.makedirs(base_dir)

train_dir = os.path.join(base_dir, 'train')
if not os.path.isdir(train_dir): 
    os.makedirs(train_dir)

test_dir = os.path.join(base_dir, 'test')
if not os.path.isdir(test_dir):
    os.makedirs(test_dir)

for classes in classes_list:
    train_split = os.path.join(train_dir, str(classes))
    if not os.path.isdir(train_split):
        os.mkdir(train_split)
    test_split = os.path.join(test_dir, str(classes))
    if not os.path.isdir(test_split):
        os.mkdir(test_split)
    

In [4]:
def what_is_this(data, file):
    target = -1

    # 마스크 여부 분류
    if 'incorrect' in file:
        target += 7
    elif 'mask' in file:
        target += 1
    else: # not wear
        target += 13

    # 성별 분류
    if data['gender'] == 'male':
        target += 0
    else: # female
        target += 3

    # 나이 분류
    if data['age'] < 30:
        target += 0
    elif data['age'] >= 60:
        target += 2
    else: # 30 ~ 60
        target += 1
    
    return target


In [5]:
def ratio_splitter(identifier, test_ratio):
    return crc32(np.string_(identifier)) & 0xffffffff < test_ratio * 2 ** 32

In [15]:
data_infos = pd.read_csv("./input/data/train/train.csv")
train_counter = {}
test_counter = {}

for i in range(18):
    train_counter[str(i)] = 0
    test_counter[str(i)] = 0

for row in data_infos.iterrows():
    data_num, data = row[0], row[1]

    # 목표 디렉토리 설정
    path = os.path.join(original_dataset_dir, data['path'])
    # 디렉토리 리스트 가져오기
    fnames = os.listdir(path)
    # 데이터 분활하기
    test_set_check = ratio_splitter(path, 0.15)
    
    for file in fnames:
        if file[0] == '.':
            continue

        target_class = what_is_this(data, file)
        target_path = "{0:02d}".format(target_class)
        
        src = os.path.join(path, file)

        if test_set_check:
            dst = os.path.join(os.path.join(test_dir, target_path), file)
        else: # train_set
            dst = os.path.join(os.path.join(train_dir, target_path), file)

        shutil.copyfile(src, dst)

        # 파일 확장자를 추출하기 위해서
        name, extension = file.split(".") 

        if test_set_check:
            change_name = os.path.join(os.path.join(test_dir, target_path), str(test_counter[str(target_class)]) + "." + extension)
            test_counter[str(target_class)] += 1 
        else: # train_set
            change_name = os.path.join(os.path.join(train_dir, target_path), str(train_counter[str(target_class)]) + "." + extension)
            train_counter[str(target_class)] += 1 
            
        shutil.move(dst, change_name)



In [7]:
transform_base = transforms.Compose([transforms.Resize([128, 86]),transforms.ToTensor()])
normalize_calc = ImageFolder(root = "./splitted/train", transform = transform_base)

In [11]:
meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x,_ in normalize_calc]
stdRGB = [np.std(x.numpy(), axis=(1,2)) for x,_ in normalize_calc]

meanR = np.mean([m[0] for m in meanRGB])
meanG = np.mean([m[1] for m in meanRGB])
meanB = np.mean([m[2] for m in meanRGB])

stdR = np.mean([s[0] for s in stdRGB])
stdG = np.mean([s[1] for s in stdRGB])
stdB = np.mean([s[2] for s in stdRGB])

print(meanR, meanG, meanB)
print(stdR, stdG, stdB)

0.5602434 0.5241528 0.5015032
0.2273327 0.23729324 0.24018635


In [5]:
meanR, meanG, meanB = 0.5602434, 0.5241528, 0.5015032
stdR, stdG, stdB = 0.2273327, 0.23729324, 0.24018635

In [6]:
new_transform = transforms.Compose([transforms.Resize([128, 86]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([meanR, meanG, meanB], [stdR, stdG, stdB])
    ])


In [7]:
def plot_results(list_of_epochs, list_of_train_losses, list_of_train_accuracies, list_of_val_accuracies):
    plt.figure(figsize=(20, 9))
    plt.subplot(1, 2, 1)
    plt.plot(list_of_epochs, list_of_train_losses, label='training loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(list_of_epochs, list_of_train_accuracies, label='training accuracy')
    plt.plot(list_of_epochs, list_of_val_accuracies, label='validation accuracy')
    plt.legend()
    if not os.path.isdir('./result_plots'):
        os.makedirs('./result_plots')
    plt.savefig('./result_plots/accuracy_plot_per_epoch.jpg')
    plt.close()

### Training routine

In [8]:
batch_size = 64

In [9]:
def set_lr(optim, epoch_num, lrate):
    """adjusts lr to starting lr thereafter reduced by 10% at every 20 epochs"""
    lrate = lrate * (0.1 ** (epoch_num // 20))
    for params in optim.param_groups:
        params['lr'] = lrate

In [10]:
def train(model, train_dataloader, optim, loss_func, epoch_num, lrate):
    model.train()
    loop_iter = 0
    training_loss = 0
    training_accuracy = 0
    for training_data, training_label in train_dataloader:
        set_lr(optim, epoch_num, lrate)
        training_data, training_label =\
            training_data.to(device), training_label.to(device)
        optim.zero_grad()
        pred_raw = model(training_data)
        curr_loss = loss_func(pred_raw, training_label)
        curr_loss.backward()
        optim.step()
        training_loss += curr_loss.data
        pred = pred_raw.data.max(1)[1]

        curr_accuracy =\
            float(pred.eq(training_label.data).sum()) * 100. / len(training_data) 
        training_accuracy += curr_accuracy
        loop_iter += 1
        if loop_iter % 100 == 0:
            print(
                f"epoch {epoch_num}, loss: {curr_loss.data}, accuracy: {curr_accuracy}")

    data_size = len(train_dataloader.dataset) // batch_size
    return training_loss / data_size, training_accuracy / data_size

### Accuracy metric

In [11]:
def accuracy(model, test_data_loader):
    model.eval()
    success = 0
    with torch.no_grad():
        for test_data, test_label in test_data_loader:
            test_data, test_label = test_data.to(device), test_label.to(device)
            pred_raw = model(test_data)
            pred = pred_raw.data.max(1)[1]
            success += pred.eq(test_label.data).sum()

    return float(success) * 100. / len(test_data_loader.dataset)

In [12]:
def load_dataset(batch_size):
    train_dataset = ImageFolder(root = "./splitted/train", transform = new_transform)
    val_dataset = ImageFolder(root = "./splitted/test", transform = new_transform)

    train_dataloader = DataLoader(train_dataset, batch_size= batch_size, shuffle=True, num_workers=4)
    test_dataloader = DataLoader(val_dataset, batch_size= batch_size, shuffle = True, num_workers=4)
    
    return train_dataloader, test_dataloader
train_dataloader, test_dataloader = load_dataset(batch_size)

### Graph Class def

In [13]:
class RndGraph(object):
    def __init__(self, num_nodes, graph_probability, nearest_neighbour_k=4,
                 num_edges_attach=5):
        self.num_nodes = num_nodes
        self.graph_probability = graph_probability
        self.nearest_neighbour_k = nearest_neighbour_k
        self.num_edges_attach = num_edges_attach

    def make_graph_obj(self):
        graph_obj = nx.random_graphs.connected_watts_strogatz_graph(
            self.num_nodes, self.nearest_neighbour_k, self.graph_probability)
        return graph_obj

    def get_graph_config(self, graph_obj):
        incoming_edges = {}
        incoming_edges[0] = []
        node_list = [0]
        last = []
        for n in graph_obj.nodes():
            neighbor_list = list(graph_obj.neighbors(n))
            neighbor_list.sort()

            edge_list = []
            passed_list = []
            for nbr in neighbor_list:
                if n > nbr:
                    edge_list.append(nbr + 1)
                    passed_list.append(nbr)
            if not edge_list:
                edge_list.append(0)
            incoming_edges[n + 1] = edge_list
            if passed_list == neighbor_list:
                last.append(n + 1)
            node_list.append(n + 1)
        incoming_edges[self.num_nodes + 1] = last
        node_list.append(self.num_nodes + 1)
        return node_list, incoming_edges

    def save_graph(self, graph_obj, path_to_write):
        if not os.path.isdir("cached_graph_obj"):
            os.mkdir("cached_graph_obj")
        #nx.write_yaml(graph_obj, "./cached_graph_obj/" + path_to_write)
        with open("./cached_graph_obj/" + path_to_write, 'w') as fh:
            yaml.dump(graph_obj, fh)

    def load_graph(self, path_to_read):
        #return nx.read_yaml("./cached_graph_obj/" + path_to_read)
        with open("./cached_graph_obj/" + path_to_read, 'r') as fh:
            return yaml.load(fh, Loader=yaml.Loader)

### randwire def

In [14]:
def initialize_weights(layer):
    if isinstance(layer, nn.Conv2d):
        torch.nn.init.xavier_uniform_(layer.weight)
        if layer.bias is not None:
            torch.nn.init.zeros_(layer.bias)

In [15]:
class SepConv2d(nn.Module):
    def __init__(self, input_ch, output_ch, kernel_length=3, dilation_size=1,
                 padding_size=1, stride_length=1, bias_flag=True):
        super(SepConv2d, self).__init__()
        self.conv_layer = nn.Conv2d(input_ch, input_ch, kernel_length,
                                    stride_length, padding_size, dilation_size,
                                    bias=bias_flag, groups=input_ch)
        self.pointwise_layer = nn.Conv2d(input_ch, output_ch, kernel_size=1,
                                         stride=1, padding=0, dilation=1, 
                                         groups=1, bias=bias_flag)

    def forward(self, x):
        return self.pointwise_layer(self.conv_layer(x))

In [16]:
class UnitLayer(nn.Module):
    def __init__(self, input_ch, output_ch, stride_length=1):
        super(UnitLayer, self).__init__()

        self.dropout = 0.3

        self.unit_layer = nn.Sequential(
            nn.ReLU(),
            SepConv2d(input_ch, output_ch, stride_length=stride_length),
            nn.BatchNorm2d(output_ch),
            nn.Dropout(self.dropout)
        )

    def forward(self, x):
        return self.unit_layer(x)

In [17]:
class GraphNode(nn.Module):
    def __init__(self, input_degree, input_ch, output_ch, stride_length=1):
        super(GraphNode, self).__init__()
        self.input_degree = input_degree
        if len(self.input_degree) > 1:
            self.params = nn.Parameter(torch.ones(
                len(self.input_degree), requires_grad=True))
        self.unit_layer = UnitLayer(
            input_ch, output_ch, stride_length=stride_length)

    def forward(self, *ip):
        if len(self.input_degree) > 1:
            op = (ip[0] * torch.sigmoid(self.params[0]))
            for idx in range(1, len(ip)):
                op += (ip[idx] * torch.sigmoid(self.params[idx]))
            return self.unit_layer(op)
        else:
            return self.unit_layer(ip[0])

In [18]:
class RandWireGraph(nn.Module):
    def __init__(self, num_nodes, graph_prob, input_ch, output_ch, train_mode,
                 graph_name):
        super(RandWireGraph, self).__init__()
        self.num_nodes = num_nodes
        self.graph_prob = graph_prob
        self.input_ch = input_ch
        self.output_ch = output_ch
        self.train_mode = train_mode
        self.graph_name = graph_name

        # get graph nodes and in edges
        rnd_graph_node = RndGraph(self.num_nodes, self.graph_prob)
        if self.train_mode is True:
            print("train_mode: ON")
            rnd_graph = rnd_graph_node.make_graph_obj()
            self.node_list, self.incoming_edge_list =\
                rnd_graph_node.get_graph_config(rnd_graph)
            rnd_graph_node.save_graph(rnd_graph, graph_name)
        else:
            rnd_graph = rnd_graph_node.load_graph(graph_name)
            self.node_list, self.incoming_edge_list =\
                rnd_graph_node.get_graph_config(rnd_graph)

        # define input Node
        self.list_of_modules = nn.ModuleList(
            [GraphNode(self.incoming_edge_list[0], self.input_ch,
                       self.output_ch, stride_length=2)])
        # define the rest Node
        self.list_of_modules.extend(
            [GraphNode(self.incoming_edge_list[n], self.output_ch,
                       self.output_ch) for n in self.node_list if n > 0])

    def forward(self, x):
        mem_dict = {}
        # start vertex
        op = self.list_of_modules[0].forward(x)
        mem_dict[0] = op

        # the rest vertex
        for n in range(1, len(self.node_list) - 1):
            # print(node, self.in_edges[node][0], self.in_edges[node])
            if len(self.incoming_edge_list[n]) > 1:
                op = self.list_of_modules[n].forward(
                    *[mem_dict[incoming_vtx] for incoming_vtx
                      in self.incoming_edge_list[n]])
            else:
                op = self.list_of_modules[n].forward(
                    mem_dict[self.incoming_edge_list[n][0]])
            mem_dict[n] = op
            
        op = mem_dict[self.incoming_edge_list[self.num_nodes + 1][0]]
        for incoming_vtx in range(
            1, len(self.incoming_edge_list[self.num_nodes + 1])):
            op += mem_dict[
                self.incoming_edge_list[self.num_nodes + 1][incoming_vtx]]
        return op / len(self.incoming_edge_list[self.num_nodes + 1])

In [19]:
class RandWireNNModel(nn.Module):
    def __init__(self, num_nodes, graph_prob, input_ch, output_ch, train_mode):
        super(RandWireNNModel, self).__init__()
        self.num_nodes = num_nodes
        self.graph_prob = graph_prob
        self.input_ch = input_ch
        self.output_ch = output_ch
        self.train_mode = train_mode
        self.dropout = 0.3
        self.class_num = 18
            
        self.conv_layer_1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=self.output_ch,
                      kernel_size=3, padding=1),
            nn.BatchNorm2d(self.output_ch),
        )

        self.conv_layer_2 = nn.Sequential(
            RandWireGraph(self.num_nodes, self.graph_prob, self.input_ch,
                          self.output_ch*2, self.train_mode, 
                          graph_name="conv_layer_2")
        )
        self.conv_layer_3 = nn.Sequential(
            RandWireGraph(self.num_nodes, self.graph_prob, self.input_ch*2,
                          self.output_ch*4, self.train_mode, 
                          graph_name="conv_layer_3")
        )
        self.conv_layer_4 = nn.Sequential(
            RandWireGraph(self.num_nodes, self.graph_prob, self.input_ch*4,
                          self.output_ch*8, self.train_mode, 
                          graph_name="conv_layer_4")
        )

        self.classifier_layer = nn.Sequential(
            nn.Conv2d(in_channels=self.input_ch*8, out_channels=1280,
                      kernel_size=1),
            nn.BatchNorm2d(1280)
        )

        self.output_layer = nn.Sequential(
            nn.Dropout(self.dropout),
            nn.Linear(1280, self.class_num)
        )

    def forward(self, x):
        x = self.conv_layer_1(x)
        x = self.conv_layer_2(x)
        x = self.conv_layer_3(x)
        x = self.conv_layer_4(x)
        x = self.classifier_layer(x)

        # global average pooling
        _, _, h, w = x.size()
        x = F.avg_pool2d(x, kernel_size=[h, w])
        x = torch.squeeze(x)
        x = self.output_layer(x)

        return x

In [20]:
num_epochs = 20
graph_probability = 0.7
node_channel_count = 64
num_nodes = 16
lrate = 0.1
train_mode = True

In [21]:
rand_wire_model = RandWireNNModel(num_nodes, graph_probability,
                   node_channel_count, node_channel_count, train_mode).to(device)

optim_module = optim.SGD(rand_wire_model.parameters(), lr=lrate,
                         weight_decay=1e-4, momentum=0.8)
loss_func = nn.CrossEntropyLoss().to(device)

epochs = []
test_accuracies = []
training_accuracies = []
training_losses = []
best_test_accuracy = 0

start_time = time.time()
for ep in range(1, num_epochs + 1):
    epochs.append(ep)
    training_loss, training_accuracy = train(rand_wire_model, train_dataloader,
                                             optim_module, loss_func, ep, lrate)
    test_accuracy = accuracy(rand_wire_model, test_dataloader)
    test_accuracies.append(test_accuracy)
    training_losses.append(training_loss.cpu())
    training_accuracies.append(training_accuracy)
    print('test acc: {0:.2f}%, best test acc: {1:.2f}%'.format(
        test_accuracy, best_test_accuracy))

    if best_test_accuracy < test_accuracy:
        model_state = {
            'model': rand_wire_model.state_dict(),
            'accuracy': test_accuracy,
            'ep': ep,
        }
        if not os.path.isdir('model_checkpoint'):
            os.mkdir('model_checkpoint')
        model_filename = "ch_count_" + str(node_channel_count) + "_prob_" +\
                          str(graph_probability)
        torch.save(model_state,
                   './model_checkpoint/' + model_filename + 'ckpt.t7')
        best_test_accuracy = test_accuracy
        plot_results(epochs, training_losses, training_accuracies,
                     test_accuracies)
    print("model train time: ", time.time() - start_time)

print(rand_wire_model)
torch.save(rand_wire_model, 'efficientnet.pt')

train_mode: ON
train_mode: ON
train_mode: ON


RuntimeError: CUDA out of memory. Tried to allocate 160.00 MiB (GPU 0; 31.75 GiB total capacity; 30.34 GiB already allocated; 141.50 MiB free; 30.48 GiB reserved in total by PyTorch)

### test model

In [ ]:
def num_model_params(model_obj):
    num_params = 0
    for l in list(model_obj.parameters()):
        l_p = 1
        for p in list(l.size()):
            l_p *= p
        num_params += l_p
    return num_params
print("total model params: ", num_model_params(rand_wire_model))

In [29]:
if os.path.exists("./model_checkpoint"):
    rand_wire_nn_model = RandWireNNModel(num_nodes, graph_probability, node_channel_count, node_channel_count,
                                         train_mode=False).to(device)
    model_filename = "ch_count_" + str(node_channel_count) + "_prob_" + str(graph_probability)
    model_checkpoint = torch.load('./model_checkpoint/' + model_filename + 'ckpt.t7')
    rand_wire_nn_model.load_state_dict(model_checkpoint['model'])
    last_ep = model_checkpoint['ep']
    best_model_accuracy = model_checkpoint['accuracy']
    print(f"best model accuracy: {best_model_accuracy}%, last epoch: {last_ep}")

    rand_wire_nn_model.eval()
    success = 0
    for test_data, test_label in test_dataloader:
        test_data, test_label = test_data.to(device), test_label.to(device)
        pred_raw = rand_wire_nn_model(test_data)
        pred = pred_raw.data.max(1)[1]
        print(pred)
        success += pred.eq(test_label.data).sum()
    print(f"test accuracy: {float(success) * 100. / len(test_dataloader.dataset)} %")

else:
    assert False, "File not found. Please check again."

best model accuracy: 84.74025974025975%, last epoch: 20
tensor([ 3,  0,  1, 15,  3,  4,  3,  1,  4,  3,  0,  0,  0,  0,  0,  3, 10,  7,
         1,  4,  9,  3,  0,  1,  0,  4,  5,  0,  1,  4,  4,  4,  1,  4, 16,  4,
        17,  3,  1,  3,  0,  7,  7,  9,  1,  4, 15, 17,  5,  1,  3,  4,  0,  3,
         3,  4,  4,  3,  4,  4,  4,  0,  3,  4], device='cuda:0')
tensor([ 3,  3,  0,  4,  5,  4,  1,  0,  3,  4,  0,  4,  0,  0,  4,  4,  0,  1,
         6,  1, 15,  2,  1,  1,  0,  0,  0,  4,  6, 16,  4, 13,  4,  3,  9, 10,
         4, 15, 15,  1,  3,  0,  3,  0, 16,  4, 16,  0,  6,  3,  4, 12,  0,  4,
         0,  4,  3,  3,  3,  3, 16,  3,  9,  0], device='cuda:0')
tensor([ 3,  4,  6,  1,  6,  4,  6,  1,  3,  3,  3, 12,  4,  0,  9,  0,  3,  4,
        12,  4, 12,  3,  3,  4, 16, 10,  4,  4,  1, 13,  1,  4,  3, 13,  3,  1,
         1,  1,  4, 16, 16, 13,  0,  4,  1, 16,  3,  3,  0,  6,  4,  4,  1,  1,
         4,  0, 13, 10,  4,  4,  2,  3,  0,  4], device='cuda:0')
tensor([ 1,  0,  4, 15,  4

In [23]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
check_dir = './input/data/eval'

In [24]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [28]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(check_dir, 'info.csv'))
image_dir = os.path.join(check_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform_test = transforms.Compose([transforms.Resize([128, 86]),
    transforms.ToTensor(),
    transforms.Normalize([meanR, meanG, meanB], [stdR, stdG, stdB])
    ])


dataset_test = TestDataset(image_paths, transform_test)

loader_test = DataLoader(
    dataset_test,
    shuffle=False
)
rand_wire_nn_model = RandWireNNModel(num_nodes, graph_probability, node_channel_count, node_channel_count,
                                        train_mode=False).to(device)

model_filename = "ch_count_" + str(node_channel_count) + "_prob_" + str(graph_probability)
model_checkpoint = torch.load('./model_checkpoint/' + model_filename + 'ckpt.t7')
rand_wire_nn_model.load_state_dict(model_checkpoint['model'])
last_ep = model_checkpoint['ep']
best_model_accuracy = model_checkpoint['accuracy']
print(f"best model accuracy: {best_model_accuracy}%, last epoch: {last_ep}")

rand_wire_nn_model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader_test:
    with torch.no_grad():
        images = images.to(device)
        pred_raw = rand_wire_nn_model(test_data)
        pred = pred_raw.data.max(1)[1]
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(check_dir, 'submission.csv'), index=False)
print('inference is done!')

best model accuracy: 84.74025974025975%, last epoch: 20


ValueError: Length of values (252000) does not match length of index (12600)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as img

index = np.random.randint(0, len(image_paths))
image = img.imread(image_paths[index])
plt.title("label : %s" % all_predictions[index])
plt.imshow(image)
plt.show()